## Load 

In [3]:
import json
opp = '../data/pd_gy/all_1.json'
with open(opp,'r') as f:
    op = json.load(f)

opf = pd.DataFrame(op)

In [58]:
import pandas as pd
op2p = './data/4filter_op.csv'
op2 = pd.read_csv(op2p)

op2.KEY = op2.KEY.astype('str')

op2 = op2[['KEY','anesthesia_type','dept','date','pt_id','pt_id_date']]

In [66]:
len(op2),len(op),len(op2m)

(662, 1766, 661)

## Merge v1 and v2 

In [65]:
op2m = pd.merge(op2,opf,how='inner',left_on='KEY',right_on='key')

In [69]:
from utils import num2class
op2m['nrs2'] = op2m.nrs.map(num2class)

## Save as json 

In [82]:
op2mj = op2m.apply(lambda x:x.to_dict(),axis=1).to_list()

with open('../data/pd_gy/all_2.json','w') as f:
    json.dump(op2mj,f)

In [93]:
print(op2m.nrs2.value_counts().to_markdown())

|    |   nrs2 |
|---:|-------:|
|  0 |    545 |
|  1 |    116 |


In [11]:
op2m.to_csv('../data/pd_gy/all_2.csv',index=False)

In [1]:
import pandas as pd
op2m = pd.read_csv('../data/pd_gy/all_2.csv')

In [2]:
from utils import num2class
op2m['nrs2_1'] = op2m.nrs.apply(lambda x : num2class(x,no_mild=[0,1,2]) )

In [5]:
op2m.nrs.value_counts().to_dict()

{2: 231, 3: 218, 1: 74, 4: 68, 5: 25, 0: 22, 6: 10, 7: 9, 9: 2, 8: 2}

### df2json

In [ ]:
jf = df.apply(lambda x:x.to_dict(),axis=1).to_list()

## v4 

In [2]:
import pandas as pd
import json
opp = '../data/pd_gy/all_3.json'
with open(opp,'r') as f:
    op = json.load(f)

opf = pd.DataFrame(op)

In [3]:
opf['or_fname'] = opf['or_path'].map(lambda x: x[0].split('\\')[-1].split('.')[0] )

In [9]:

black_list = ['01826958_PDOR1_210421_124500_1',
              '01853529_PDOR1_210419_083500_1',
              '01876359_PDOR2_210602_083200_1',
              '01656320_PDOR2_210526_091300_1',
              '01618678_PDOR1_210615_083000_1',
              '01159584_PDOR1_211025_083400_1',
              '00578148_PDOR2_210805_111500_1',
              '01853408_PDOR1_211115_083100_1',
              '01906040_PDOR1_210831_110500_1',
              '00435676_PDREC02_210902_093500_1', # dataset 
              '01060037_PDOR2_211206_102000_1', #  Filling missing values error
              '01929835_PDOR2_211102_093000_1',
              '01392754_PDOR2_220111_105500_1'
             ]
# black_list = ['01405184_PDOR1_210329_113500_1','00618336_PDOR1_210329_155000_1']
indices = opf[opf['or_fname'].isin(black_list)].index
opf = opf.drop(indices)

In [10]:
len(opf)

649

In [11]:
jf = opf.apply(lambda x:x.to_dict(),axis=1).to_list()

import json 
with open('../data/pd_gy/all_3.json','w') as f:
    json.dump(jf,f)

## check shape 

In [7]:
from utils import get_sample_config
orp, recp, sample_rate, min5, item = get_sample_config()

# ppg, ecg
from dataset import get_vital
ppg_or, ecg_or   = get_vital(orp, type_='or',sample_rate=sample_rate, duration=min5)
ppg_rec, ecg_rec = get_vital(recp, type_='rec',sample_rate=sample_rate, duration=min5)

In [8]:
from preprocess import list2specgram
x = list2specgram([ppg_or, ppg_rec, ecg_or, ecg_rec],type_=['ppg','ppg','ecg','ecg'], sample_rate=sample_rate)
ppg_or, ppg_rec, ecg_or, ecg_rec = x

In [14]:
if all([sig.shape == (151,342) for sig in [ppg_or,ppg_rec,ecg_or,ecg_rec]]):
    vital = np.array([ppg_or,ecg_or,ppg_rec,ecg_rec])
else:
    vital = np.random.rand(4,151,342)
    print('\033[32m \033[41m'+f'Error: {orp}\n{recp}\n\n '+ '\033[0m')

[True, True, True, True]

In [7]:
import torch
from torch.utils.data import DataLoader
from dataset import VitalDataset
root_dir = '../data/pd_gy'
trdt  = VitalDataset(root_dir,'../data/pd_gy/all_3.json')

alldl  = torch.utils.data.DataLoader(trdt,4)